In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
import os

In [21]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))


vgg_dir = os.path.join(BASE_DIR, "models", "VGG19_trained.keras")
dn_dir = os.path.join(BASE_DIR, "models", "DenseNet_trained.keras")
val_dir = os.path.join(BASE_DIR, "data", "preprocessed_split_2","test")


vgg_model = tf.keras.models.load_model(vgg_dir)
dn_model = tf.keras.models.load_model(dn_dir)

In [23]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

val_gen_dn = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
    color_mode="grayscale"
)


val_gen_vgg = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
    color_mode="rgb"
)

Found 333 images belonging to 2 classes.


Found 333 images belonging to 2 classes.


In [24]:
def get_binary_metrics(model, generator):
    y_true = generator.classes
    y_pred_probs = model.predict(generator, verbose=0)
    
    # Convert probabilities to class labels
    if y_pred_probs.ndim > 1 and y_pred_probs.shape[1] == 1:
        y_pred = (y_pred_probs > 0.5).astype(int).flatten()
    else:
        y_pred = np.argmax(y_pred_probs, axis=1)
    
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return {"Accuracy": acc, "Precision": precision, "Recall": recall, "F1-Score": f1}

In [26]:
vgg_metrics = get_binary_metrics(vgg_model, val_gen_vgg)
dn_metrics = get_binary_metrics(dn_model, val_gen_dn)



print("Metric        | DN      | VGG19")
print("--------------------------------------")
for key in dn_metrics.keys():
    print(f"{key:13} | {dn_metrics[key]:.4f}   | {vgg_metrics[key]:.4f}")

Metric        | DN      | VGG19
--------------------------------------
Accuracy      | 0.9880   | 0.6456
Precision     | 0.9941   | 0.5950
Recall        | 0.9825   | 0.9708
F1-Score      | 0.9882   | 0.7378
